In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd
# wait = WebDriverWait(driver, 10)

In [2]:
#dat = pd.read_csv('ectreeToExtract.csv', encoding="utf-8")

In [3]:
ectree_str = "1.1.1.1"

In [4]:
driver = webdriver.Chrome()
url = 'https://www.brenda-enzymes.org/enzyme.php?ecno=' + ectree_str
driver.get(url)

KeyboardInterrupt: 

In [ ]:
#show all tables
driver.find_element(By.XPATH, '//*[@id="flatnav_sa"]').click()

In [ ]:
title = driver.find_element(By.XPATH, '//*[@id="flatheader"]/div/h1')
print(title.text)

In [ ]:
# //*[@id="tab47"] https://meet.google.com/csj-cxpp-ejv //*[@id="tab16"]
title = driver.find_element(By.XPATH, '//*[@id="tab23_head"]/div[1]')
print(title.text)

In [ ]:
chks = driver.find_elements(By.XPATH, "//*[@id='tab23']//a[@href]")

ent = 'entries'

for chk in chks:
    txt = chk.text
    if ent in txt:
        chk.click()

In [ ]:
tab1_eles = driver.find_elements(By.XPATH, '//div[@id="tab23" and @class="equal"]')

In [ ]:
table_columns = []

for tab in tab1_eles:
    
    col_list = tab.find_elements(By.CLASS_NAME, 'header')
    
    for listss in col_list:
        print(listss.text)
        table_columns.append(listss.text)

In [ ]:
final_df = pd.DataFrame(columns = table_columns)

In [ ]:
final_df

In [ ]:
tab1_eles = driver.find_elements(By.XPATH, "//div[contains(@id, 'tab23r') and contains(@class, 'cell')]")

In [ ]:
table_values = []
for tab in tab1_eles:
    try:
        link = tab.find_element(By.TAG_NAME, 'a')
        temp_link = link.get_attribute('href')
        
        # if not('showRows' or 'hideRows') in temp_link:
        if ('showRows' in temp_link or 'hideRows' in temp_link):
            #print(tab.text)
            table_values.append(tab.text)
            # print(tab.text + ', ' + temp_link)
            # table_values.append(tab.text + ',' + temp_link)
            
        else:
            # print(tab.text)
            # table_values.append(tab.text)
            
            #print(tab.text + ', ' + temp_link)
            table_values.append(tab.text + ',' + temp_link)
            
    except:
        table_values.append(tab.text)
        #print(tab.text)

In [ ]:
len(table_values)

In [ ]:
(len(table_values)/len(table_columns))

In [ ]:
for i in range(len(table_columns)):
    temp_value = table_values[i::(len(table_columns))]
    #print(temp_value)
    
    final_df[table_columns[i]] = temp_value

In [ ]:
final_df.head()

final_df.to_csv('table_for_test.csv')

#### formatting column names

In [ ]:
final_df.isnull().sum()

final_df.columns = [c.replace("\n", "_") for c in final_df.columns]

In [ ]:
final_df.head()

In [ ]:
final_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.ffill(inplace = True)

In [ ]:
final_df.head(10)

In [ ]:
final_df.dtypes

In [ ]:
http_columns = final_df.columns[final_df.apply(lambda x: x.str.contains("https")).any()]

In [ ]:
len(http_columns)

main_df = final_df.copy()

In [ ]:
for i in range(len(http_columns)):
    # print(http_columns[i])
    temp_string = http_columns[i] + ' ' + 'href'
    # new = final_df[http_columns[i]].str.split(",", n = 1, expand = True)
    new = final_df[http_columns[i]].str.rsplit(",", n=1, expand = True)
    # print(new[0])        
    # print(new[1])
    final_df[http_columns[i]] = new[0]
    final_df[temp_string]= new[1] 

final_df.head(20)

In [ ]:
js_columns = final_df.columns[final_df.apply(lambda x: x.str.contains("javascript")).any()]

In [ ]:
len(js_columns)

In [ ]:
new_columns = []

for i in range(len(js_columns)):
    print(js_columns[i])
    temp_string = js_columns[i] + ' ' + 'href'
    # new = final_df[js_columns[i]].str.split(",", n = 1, expand = True)
    new = final_df[js_columns[i]].str.rsplit(",", n = 1, expand = True)
                   
    final_df[js_columns[i]] = new[0]
    final_df[temp_string]= new[1] 
    
    new_columns.append(temp_string)

# After getting 

In [ ]:
splitword = final_df["PDB"].str.split(",", n=1, expand = True)
final_df["commondescrip"] = splitword[0]
final_df["commondescrip"]

In [ ]:
final_df

In [ ]:
final_df.head()

In [ ]:
# splitword = final_df["PDB"].str.split(",", n=1, expand = True)
# final_df["commondescrip"] = splitword[0]
# final_df["commondescrip"]

In [ ]:
len(final_df)

# explode

final_df.head(50)

final_df.tail()

In [ ]:
#final_df.to_csv('final_df.csv')

In [ ]:
len(final_df)

In [ ]:
new_columns

In [ ]:
new_columns.remove('PDB href')

In [ ]:
new_columns.remove('SCOP href')

In [ ]:
new_columns.remove('CATH href')

In [ ]:
new_columns

In [ ]:
temp_list_href = []
for i in range(len(new_columns)):
    temp_list = final_df[new_columns[i]]
    
    headers = [item for item in temp_list if item is not None]
    temp_list_href.append(headers[0])

temp_list_href = temp_list_href[:2]

In [ ]:
temp_list_href

In [ ]:
#{LINK} + 53953

resulted_link = []

for i in range(len(temp_list_href)):
    # wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "%s")]' % temp_list_href[i])))
    links = driver.find_element(By.XPATH, '//a[contains(@href, "%s")]' % temp_list_href[i])
    print(links)
    links.click()
    print(temp_list_href[i])
    
    
    driver.switch_to.window(driver.window_handles[-1])
    
    print(len(driver.window_handles))
    
    temp_link = driver.current_url
    resulted_link.append(temp_link)
    
    if(len(driver.window_handles) > 1):
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
    if(len(driver.window_handles) == 1):
        driver.back()
        driver.switch_to.window(driver.window_handles[0])

In [ ]:
resulted_link = ['https://www.brenda-enzymes.org/sequences.php?AC=A0A1E3MC83','https://www.brenda-enzymes.org/taxonomy.php?f[id][value]=470']

In [ ]:
len(final_df)

In [ ]:
final_df.head()

In [ ]:
temp_df = final_df[new_columns]

In [ ]:
len(temp_df)

In [ ]:
len(new_columns)

In [ ]:
for i in range(len(new_columns)):
    print(new_columns[i])
    new_link = resulted_link[i]
    
    for j in range(len(temp_df)):
        old_link = temp_df[new_columns[i]].iloc[[j]].values
        
        try:
            extracted_new_link_str = re.findall(r'\b\d+\b', new_link)
            
            extracted_old_link_str = re.findall(r"'(.*?)'", old_link[0], re.DOTALL) 
            
            if not extracted_new_link_str:
                extracted_new_link_str = new_link.split("=",1)[1]
                updated_final_link = new_link.replace(str(extracted_new_link_str), str(extracted_old_link_str[0]))
            else:
                updated_final_link = new_link.replace(str(extracted_new_link_str[0]), str(extracted_old_link_str[0]))
                
            print(f'old link: {old_link}')
            print(f'extracted from old link: {extracted_old_link_str[0]}')
            print(f'new link: {new_link}')
            print(f'extracted from new link: {extracted_new_link_str[0]}') 
            
            print(f'updated final link: {updated_final_link}')
            print('\n')
            
            temp_df[new_columns[i]].iloc[[j]]= updated_final_link
        
        except:
            pass

temp_df.head()

In [ ]:
for i in range(len(new_columns)):
    final_df = final_df.drop(new_columns[i], axis = 1)

final_df.head()

temp_df.tail()

In [ ]:
final_df = final_df.drop_duplicates()

In [ ]:
diff = len(temp_df) - len(final_df)

final_df.tail()

In [ ]:
temp_df.drop(temp_df.tail(diff).index,inplace = True)

In [ ]:
len(temp_df)

In [ ]:
frames = [final_df, temp_df]

In [ ]:
final_df = pd.concat([final_df, temp_df], axis = 1)

final_df.head()

In [ ]:
final_df.index = np.arange(1, len(final_df) + 1)

In [ ]:
final_df.insert(loc = 0,column = 'id',value = final_df.index)

In [ ]:
final_df.insert(loc = 1,column = 'ectree',value = ectree_str)

In [ ]:
final_df.replace(to_replace=[None], value='-', inplace=True)

In [ ]:
final_df.columns = [c.replace(" ", "_") for c in final_df.columns]

In [ ]:
final_df.columns = [c.replace("(", "") for c in final_df.columns]

In [ ]:
final_df.columns = [c.replace(")", "") for c in final_df.columns]

In [ ]:
final_df.columns = [c.replace("?", "") for c in final_df.columns]

In [ ]:
final_df.columns = [c.replace("=", "_equals_") for c in final_df.columns]

In [ ]:
final_df = final_df.replace('', np.NaN)
final_df.ffill(inplace = True)

In [ ]:
final_df

In [ ]:
temp = final_df.drop(['PDB_href', 'SCOP_href','CATH_href'], axis=1)

In [ ]:
temp['PDB href'] = 'https://www.rcsb.org/structure/' + final_df['commondescrip'].astype(str)

In [ ]:
temp['SCOP href'] = 'http://scop.berkeley.edu/pdb/code=' + final_df['commondescrip'].astype(str)

In [ ]:
temp['CATH href'] = 'http://www.cathdb.info/pdb/' + final_df['commondescrip'].astype(str)

In [ ]:
temp['PDB 3-D href'] = 'https://www.brenda-enzymes.org/Mol/ngl/index.php?pdb='+ final_df['commondescrip'].astype(str)+'&ecno=1.1.1.1'

In [ ]:
correct  = temp.drop(['commondescrip'], axis=1)

In [ ]:
#https://www.brenda-enzymes.org/Mol/ngl/index.php?pdb=1h2b&ecno=1.1.1.1

In [ ]:
correct

In [ ]:
correct.to_csv('PDB1.csv')

#### reaction_diagram process

if 'REACTION_DIAGRAM' in final_df:
    final_df['REACTION_DIAGRAM'] = final_df['REACTION_DIAGRAM_href']
    final_df.drop('REACTION_DIAGRAM_href', axis = 1, inplace = True)

In [ ]:
len(final_df)

final_df.head()

final_df.to_csv('corrected_phStability.csv')

#### Writing on mysql

In [ ]:
# import mysql.connector

In [ ]:
# mydb = mysql.connector.connect(host = 'quantumzyme.com', user = 'zummit_user', password = 'kZf-ih&[xV@2', database = 'qzBrenda', use_pure = True)

#### create table string 

In [ ]:
# import pandas as pd
# dataframe = pd.read_csv('localization.csv')

In [ ]:
# dataframe.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# dataframe

In [ ]:
# final_df.columns

In [ ]:
# dataframe.drop('id', axis = 1, inplace = True)

In [ ]:
# dataframe

In [ ]:
# combined_string = []
# for i in range(len(final_df.columns)):
#     if(len(final_df.columns[i]) <= 64):
#         temp_string = final_df.columns[i] + ' VARCHAR(500)'
#         combined_string.append(temp_string) 
#     else:
#         temp_string = final_df.columns[i][0:63] + ' VARCHAR(500)'
#         combined_string.append(temp_string)
    
# combined_string_new = ', '.join(combined_string)
# print(combined_string_new)
# print('\n')

# create_table_string = 'CREATE TABLE IF NOT EXISTS '+ final_df.columns[1] + '(id int NOT NULL AUTO_INCREMENT, ' + combined_string_new + ', PRIMARY KEY (id))'
# print(create_table_string)

In [ ]:
# mycursor = mydb.cursor()

In [ ]:
# try:
#     mycursor.execute(create_table_string)

# except:
#     pass

#### insert into table

In [ ]:
# combined_insert_string = []
# percent_s = []
# percent = '%s'
# for i in range(len(final_df.columns)):
#     if(len(final_df.columns[i]) >= 64):
#         combined_insert_string.append(final_df.columns[i][0:63])
#         percent_s.append(percent)

#     else:
#         combined_insert_string.append(final_df.columns[i])
#         percent_s.append(percent)

# print(i)
# # percent_s = percent_s[:-1]
# percent_s = ', '.join(percent_s)

# combined_insert_string = ', '.join(combined_insert_string)

# insert_string = 'INSERT INTO ' + final_df.columns[1] + '(' + combined_insert_string + ') VALUES(' + percent_s  + ')'
# print(insert_string)

In [ ]:
# df_columns = []
# for i in range(len(final_df.columns)):
#     if(len(final_df.columns[i]) >= 64):
#         df_columns.append(final_df.columns[i][0:63])
#     else:
#         df_columns.append(final_df.columns[i]) 
# df_columns

In [ ]:
# df_columns = []
# for i in range(len(final_df.columns)):
#     if(len(final_df.columns[i]) >= 64):
#         df_columns.append(final_df.columns[i][0:63])
#     else:
#         df_columns.append(final_df.columns[i]) 

# df_columns

In [ ]:
# data = tuple(df_columns)
# data

In [ ]:
# for(row, rs) in final_df.iterrows():
#     for i in range(len(df_columns)):
#         print(rs[i])
#         df_columns[i] = rs[i]
            
#     data = tuple(df_columns)
#     mycursor.execute(insert_string, data)
#     mydb.commit()

In [1]:
import mysql.connector

In [2]:
mydb = mysql.connector.connect(host = 'quantumzyme.com', user = 'zummit_user', password = 'kZf-ih&[xV@2', database = 'qzBrenda', use_pure = True)

In [3]:
mycursor = mydb.cursor()

In [4]:
import pandas as pd
dataframe = pd.read_csv('PDB1.csv')

In [5]:
dataframe

,Unnamed: 0,id,ectree,PDB,SCOP,CATH,UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href
0,1,1,1.1.1.1,"6c49, download, 3D-view",SCOPe (6c49),CATH (6c49),A0A1E3MC83,Acinetobacter baumannii,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/6c49,http://scop.berkeley.edu/pdb/code=6c49,http://www.cathdb.info/pdb/6c49,https://www.brenda-enzymes.org/Mol/ngl/index.p...
1,2,2,1.1.1.1,"1h2b, download, 3D-view",SCOPe (1h2b),CATH (1h2b),Q9Y9P9,Aeropyrum pernix,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1h2b,http://scop.berkeley.edu/pdb/code=1h2b,http://www.cathdb.info/pdb/1h2b,https://www.brenda-enzymes.org/Mol/ngl/index.p...
2,3,3,1.1.1.1,"3uko, download, 3D-view",SCOPe (3uko),CATH (3uko),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/3uko,http://scop.berkeley.edu/pdb/code=3uko,http://www.cathdb.info/pdb/3uko,https://www.brenda-enzymes.org/Mol/ngl/index.p...
3,4,4,1.1.1.1,"4gl4, download, 3D-view",SCOPe (4gl4),CATH (4gl4),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4gl4,http://scop.berkeley.edu/pdb/code=4gl4,http://www.cathdb.info/pdb/4gl4,https://www.brenda-enzymes.org/Mol/ngl/index.p...
4,5,5,1.1.1.1,"4jji, download, 3D-view",SCOPe (4jji),CATH (4jji),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4jji,http://scop.berkeley.edu/pdb/code=4jji,http://www.cathdb.info/pdb/4jji,https://www.brenda-enzymes.org/Mol/ngl/index.p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,185,1.1.1.1,"1o2d, download, 3D-view",SCOPe (1o2d),CATH (1o2d),Q9X022,Thermotoga maritima,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1o2d,http://scop.berkeley.edu/pdb/code=1o2d,http://www.cathdb.info/pdb/1o2d,https://www.brenda-enzymes.org/Mol/ngl/index.p...
185,186,186,1.1.1.1,"1vj0, download, 3D-view",SCOPe (1vj0),CATH (1vj0),Q9WYR7,Thermotoga maritima,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1vj0,http://scop.berkeley.edu/pdb/code=1vj0,http://www.cathdb.info/pdb/1vj0,https://www.brenda-enzymes.org/Mol/ngl/index.p...
186,187,187,1.1.1.1,"4cpd, download, 3D-view",SCOPe (4cpd),CATH (4cpd),B2ZRE3,Thermus sp. ATN1,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4cpd,http://scop.berkeley.edu/pdb/code=4cpd,http://www.cathdb.info/pdb/4cpd,https://www.brenda-enzymes.org/Mol/ngl/index.p...
187,188,188,1.1.1.1,"3owo, download, 3D-view",SCOPe (3owo),CATH (3owo),P0DJA2,Zymomonas mobilis,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/3owo,http://scop.berkeley.edu/pdb/code=3owo,http://www.cathdb.info/pdb/3owo,https://www.brenda-enzymes.org/Mol/ngl/index.p...


In [6]:
dataframe.drop(['Unnamed: 0','id'], axis = 1, inplace = True)

In [7]:
dataframe

,ectree,PDB,SCOP,CATH,UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href
0,1.1.1.1,"6c49, download, 3D-view",SCOPe (6c49),CATH (6c49),A0A1E3MC83,Acinetobacter baumannii,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/6c49,http://scop.berkeley.edu/pdb/code=6c49,http://www.cathdb.info/pdb/6c49,https://www.brenda-enzymes.org/Mol/ngl/index.p...
1,1.1.1.1,"1h2b, download, 3D-view",SCOPe (1h2b),CATH (1h2b),Q9Y9P9,Aeropyrum pernix,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1h2b,http://scop.berkeley.edu/pdb/code=1h2b,http://www.cathdb.info/pdb/1h2b,https://www.brenda-enzymes.org/Mol/ngl/index.p...
2,1.1.1.1,"3uko, download, 3D-view",SCOPe (3uko),CATH (3uko),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/3uko,http://scop.berkeley.edu/pdb/code=3uko,http://www.cathdb.info/pdb/3uko,https://www.brenda-enzymes.org/Mol/ngl/index.p...
3,1.1.1.1,"4gl4, download, 3D-view",SCOPe (4gl4),CATH (4gl4),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4gl4,http://scop.berkeley.edu/pdb/code=4gl4,http://www.cathdb.info/pdb/4gl4,https://www.brenda-enzymes.org/Mol/ngl/index.p...
4,1.1.1.1,"4jji, download, 3D-view",SCOPe (4jji),CATH (4jji),Q96533,Arabidopsis thaliana,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4jji,http://scop.berkeley.edu/pdb/code=4jji,http://www.cathdb.info/pdb/4jji,https://www.brenda-enzymes.org/Mol/ngl/index.p...
...,...,...,...,...,...,...,...,...,...,...,...,...
184,1.1.1.1,"1o2d, download, 3D-view",SCOPe (1o2d),CATH (1o2d),Q9X022,Thermotoga maritima,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1o2d,http://scop.berkeley.edu/pdb/code=1o2d,http://www.cathdb.info/pdb/1o2d,https://www.brenda-enzymes.org/Mol/ngl/index.p...
185,1.1.1.1,"1vj0, download, 3D-view",SCOPe (1vj0),CATH (1vj0),Q9WYR7,Thermotoga maritima,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/1vj0,http://scop.berkeley.edu/pdb/code=1vj0,http://www.cathdb.info/pdb/1vj0,https://www.brenda-enzymes.org/Mol/ngl/index.p...
186,1.1.1.1,"4cpd, download, 3D-view",SCOPe (4cpd),CATH (4cpd),B2ZRE3,Thermus sp. ATN1,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/4cpd,http://scop.berkeley.edu/pdb/code=4cpd,http://www.cathdb.info/pdb/4cpd,https://www.brenda-enzymes.org/Mol/ngl/index.p...
187,1.1.1.1,"3owo, download, 3D-view",SCOPe (3owo),CATH (3owo),P0DJA2,Zymomonas mobilis,https://www.brenda-enzymes.org/sequences.php?A...,https://www.brenda-enzymes.org/taxonomy.php?f[...,https://www.rcsb.org/structure/3owo,http://scop.berkeley.edu/pdb/code=3owo,http://www.cathdb.info/pdb/3owo,https://www.brenda-enzymes.org/Mol/ngl/index.p...


In [8]:
for(row, rs) in dataframe.iterrows():
    ectree= rs[0]
    PDB= rs[1]
    SCOP  = rs[2]
    CATH = rs[3]
    UNIPROT = rs[4]
    ORGANISM = rs[5]
    UNIPROT_href = rs[6]
    ORGANISM_href = rs[7]
    PDB_href= rs[8]
    SCOP_href = rs[9]
    CATH_href = rs[10]
    PDB_3_D_href = rs[11]
    
    insert_query = "INSERT INTO qzBrenda.PDB_and_Structure_Links(ectree, PDB,SCOP,CATH, UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href) VALUES( %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s)"
    data = (ectree, PDB,SCOP,CATH, UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href)
    
    mycursor.execute(insert_query, data)
    mydb.commit()

In [9]:
# for(row, rs) in dataframe.iterrows():
#     ectree= rs[0]
#     PDB= rs[1]
#     SCOP  = rs[2]
#     CATH = rs[3]
#     UNIPROT = rs[4]
#     ORGANISM = rs[5]
#     UNIPROT_href = rs[6]
#     ORGANISM_href = rs[7]
#     PDB_href= rs[8]
#     SCOP_href = rs[9]
#     CATH_href = rs[10]
#     PDB_3_D_href = rs[11]
    
#     insert_query = "INSERT INTO qzBrenda.PDB_and_Structure_Links(ectree, PDB,SCOP,CATH, UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href) VALUES( %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s)"
#     data = (ectree, PDB,SCOP,CATH, UNIPROT,ORGANISM,UNIPROT_href,ORGANISM_href,PDB_href,SCOP_href,CATH_href,PDB_3_D_href)
    
#     mycursor.execute(insert_query, data)
#     mydb.commit()

In [ ]:
# mydb.close()